In [6]:
import cv2
import os
import numpy as np
from pathlib import Path
import shutil

VIDEOS_PATH = '../videos'
DATASET_PATH = '../dataset'
CLASSES = ['entrada', 'salida']
CLIP_LENGTH = 32
RESIZE = (224, 224)

def limpiar_directorio():
    if os.path.exists(DATASET_PATH):
        shutil.rmtree(DATASET_PATH)
    for clase in CLASSES:
        os.makedirs(os.path.join(DATASET_PATH, clase), exist_ok=True)

def resize_con_padding(frame, target_size=(224, 224)):
    h, w = frame.shape[:2]
    scale = min(target_size[0] / h, target_size[1] / w)
    new_w, new_h = int(w * scale), int(h * scale)
    resized = cv2.resize(frame, (new_w, new_h))

    top = (target_size[0] - new_h) // 2
    bottom = target_size[0] - new_h - top
    left = (target_size[1] - new_w) // 2
    right = target_size[1] - new_w - left

    padded = cv2.copyMakeBorder(resized, top, bottom, left, right,
                                 borderType=cv2.BORDER_CONSTANT, value=[0, 0, 0])
    return padded

def extraer_clip_uniforme(video_path, clase):
    cap = cv2.VideoCapture(str(video_path))
    frames = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Corregir inversión horizontal si es necesario
        frame = cv2.flip(frame, 0)  # ← flip vertical

        # Resize con padding para mantener proporciones
        frame = resize_con_padding(frame, RESIZE)
        frames.append(frame)
    cap.release()

    total = len(frames)
    if total < CLIP_LENGTH:
        print(f"⚠️ {video_path.name} tiene solo {total} frames. Se omite.")
        return

    # Seleccionar 16 frames uniformemente distribuidos
    indices = np.linspace(0, total - 1, num=CLIP_LENGTH, dtype=int)
    clip = [frames[i] for i in indices]

    clip_dir = Path(DATASET_PATH) / clase / video_path.stem
    os.makedirs(clip_dir, exist_ok=True)
    for i, f in enumerate(clip):
        cv2.imwrite(str(clip_dir / f"{i:03d}.jpg"), f)

def construir_dataset():
    limpiar_directorio()
    video_dir = Path(VIDEOS_PATH)
    for video_file in video_dir.glob("*.mp4"):
        nombre = video_file.stem.lower()
        clase = next((cls for cls in CLASSES if cls in nombre), None)
        if not clase:
            print(f"Omitiendo {video_file.name}: clase no detectada.")
            continue
        print(f"✅ Procesando {video_file.name} como '{clase}'")
        extraer_clip_uniforme(video_file, clase)

if __name__ == "__main__":
    construir_dataset()


✅ Procesando entrada_045.mp4 como 'entrada'
✅ Procesando entrada_010.mp4 como 'entrada'
✅ Procesando entrada_002.mp4 como 'entrada'
✅ Procesando salida_013.mp4 como 'salida'
✅ Procesando salida_023.mp4 como 'salida'
✅ Procesando entrada_026.mp4 como 'entrada'
✅ Procesando salida_008.mp4 como 'salida'
✅ Procesando salida_031.mp4 como 'salida'
✅ Procesando entrada_017.mp4 como 'entrada'
✅ Procesando entrada_021.mp4 como 'entrada'
✅ Procesando entrada_040.mp4 como 'entrada'
✅ Procesando salida_024.mp4 como 'salida'
✅ Procesando salida_019.mp4 como 'salida'
✅ Procesando entrada_027.mp4 como 'entrada'
✅ Procesando entrada_005.mp4 como 'entrada'
✅ Procesando salida_047.mp4 como 'salida'
✅ Procesando entrada_047.mp4 como 'entrada'
✅ Procesando salida_038.mp4 como 'salida'
✅ Procesando salida_033.mp4 como 'salida'
✅ Procesando salida_045.mp4 como 'salida'
✅ Procesando salida_030.mp4 como 'salida'
✅ Procesando entrada_036.mp4 como 'entrada'
✅ Procesando entrada_006.mp4 como 'entrada'
✅ Procesan